In [3]:
clean_nq_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Datasets\\NATURAL_QUESTIONS\\clean_nq_format.jsonl'


In [8]:
import jsonlines
import pdb
import re
import pandas as pd
import math

In [15]:
def chunkstring(text,tokens_length = 200): 
    text_tokens = [token for token in text.split(' ')]
    tokens_length = 200
    n_chunks = math.ceil(len(text_tokens)/tokens_length)
    chunck_no_context = [' '.join(text_tokens[(0+i*tokens_length): (tokens_length-1+i*tokens_length)]) for i in range(0, n_chunks)]

    return chunck_no_context

In [38]:
def nq_dict_to_df():
    
    df_doc_text = pd.DataFrame(columns=['ID','DOC_TEXT'])
    df_long_answer = pd.DataFrame(columns=['ID','LONG_ANSWER','START_LA','END_LA'])
    df_short_answer = pd.DataFrame(columns=['ID','SHORT_ANSWER','START_SA','END_SA'])
    df_question = pd.DataFrame(columns=['ID','QUESTION'])
    df_infos = pd.DataFrame(columns=['ID','DOCUMENT_URL','YES_NO_ANSWER'])
    
    with jsonlines.open(clean_nq_path) as reader:
        for row in reader:
            
            # Document Text
            doctext_chuncks = chunkstring(row['document_text'])
            for chunk in doctext_chuncks:
                df_doc_text = df_doc_text.append(pd.Series([row['example_id'],chunk], index=df_doc_text.columns), ignore_index=True)
            
            #long Answer
            start_token = row['annotations'][0]['long_answer']['start_token']
            end_token = row['annotations'][0]['long_answer']['end_token']
            long_answer = " ".join(row['document_text'].split(" ")[start_token:end_token])
            long_answer_chunks = chunkstring(long_answer)
            for chunk in long_answer_chunks:
                df_long_answer = df_long_answer.append(pd.Series([row['example_id'],chunk,start_token,end_token], index=df_long_answer.columns), ignore_index=True)
            
            # Short Answers
            short_answers = row['annotations'][0]['short_answers']
            for sa in short_answers:
                start_token = sa['start_token']
                end_token = sa['end_token']
                short_answer = " ".join(row['document_text'].split(" ")[start_token:end_token])
                df_short_answer = df_short_answer.append(pd.Series([row['example_id'],short_answer,start_token,end_token], index=df_short_answer.columns), ignore_index=True)

            # Questions
            df_question = df_question.append(pd.Series([row['example_id'],row['question_text']], index=df_question.columns), ignore_index=True)
             
            # Infos
            df_infos = df_infos.append(pd.Series([row['example_id'],row['document_url'],row['annotations'][0]['yes_no_answer']], index=df_infos.columns), ignore_index=True)
    
        
    return df_doc_text,df_long_answer,df_short_answer,df_question,df_infos
     

In [35]:
def save_in_tables(df_doc_text,df_long_answer,df_short_answer,df_question,df_infos):
    # Document Text
    compression_opts = dict(method='zip', archive_name='nq_doc_text.csv') 
    df_doc_text.to_csv('nq_doc_text.zip', index=False,compression=compression_opts)

    # long Answer
    compression_opts = dict(method='zip', archive_name='nq_long_answer.csv') 
    df_long_answer.to_csv('nq_long_answer.zip', index=False,compression=compression_opts)
    
    # Short Answers
    compression_opts = dict(method='zip', archive_name='nq_short_answer.csv') 
    df_short_answer.to_csv('nq_short_answer.zip', index=False,compression=compression_opts)
    
    # Questions
    compression_opts = dict(method='zip', archive_name='nq_question.csv') 
    df_question.to_csv('nq_question.zip', index=False,compression=compression_opts)
    
    # Infos
    compression_opts = dict(method='zip', archive_name='nq_infos.csv') 
    df_infos.to_csv('nq_infos.zip', index=False,compression=compression_opts)  

In [39]:
df_doc_text,df_long_answer,df_short_answer,df_question,df_infos = nq_dict_to_df()

In [40]:
save_in_tables(df_doc_text,df_long_answer,df_short_answer,df_question,df_infos)